## Load and Check data
We will load datasets, inspect them for any errors, fix if any with explanations.

In [ ]:
# necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#load the data files
try:
    rides=pd.read_csv('../input/taxi-trip-analysis/project_sql_result_01.csv')
    nbhoods=pd.read_csv('../input/taxi-trip-analysis/project_sql_result_04.csv')
    to_airport=pd.read_csv('../input/taxi-trip-analysis/project_sql_result_07.csv')
    print('All files has been imported!')
except:
    print('Something went wrong, check paths, separators etc!')

In [ ]:
# get familiarized with content and data types of datasets 
for df in [rides, nbhoods, to_airport]:
    display(df.head(3))
    df.info()
    print()
    print('='*50)

In [ ]:
#change data type of to_airport['start_ts'] to datetime
to_airport['start_ts'] = pd.to_datetime(to_airport['start_ts'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

#check the result
to_airport.info()

In [ ]:
# check for missing and duplicated values in each dataset
for df in [rides, nbhoods, to_airport]:
    display(df.isna().sum())
    print('Duplicated number of rows:', df.duplicated().sum())
    print('='*50)

### Conclusion
All three dataset has been successfully loaded and been got familiarized. Datatypes of columns were correct, except the `start_ts` column of the to_airport dataset. It was in *object* type and then converted to *datetime* type. Datasets have been check for missing and duplicate values. No significant issues were observed other than duplicates in the to_airport table. However, we don't have a trip_id column, therefore we can not exactly say if these rows are duplicated or separate observations with equal values. Hence, it was decided to keep them!

## Explaratory data analysis
We will make graphs: to analyze companies by number of rides and neighborhoods by number of drop-offs.

In [ ]:
#prepare data: top companies with most trips on November 15-16, 2017
data=rides.sort_values('trips_amount', ascending=False)[:10]

#create fig object
plt.figure(figsize=(12,8))

#horizontal bar chart showing number of trips per company
sns.barplot(x='trips_amount', y='company_name', data=data, palette='inferno')

#customize the plot
plt.title('Top 10 companies with highest number of trips on November 15-16, 2017', fontsize=16, color='r')
plt.xlabel('Number of trips', fontsize=14, color='k')
plt.ylabel('Name of Company', fontsize=14, color='k')
plt.grid(b=True, which='both', color='b', linestyle='--', alpha=0.1)
plt.minorticks_on()
plt.show()

In [ ]:
#prepare data: top neighborhoods with most drop-offs in November 2017
data=nbhoods.sort_values('average_trips', ascending=False)[:10]

#create figure object
plt.figure(figsize=(12,8))

#h.bar chart showing top neighborhoods with highest avg. drop-offs
sns.barplot(y='dropoff_location_name', x='average_trips', data=data, palette='inferno')

#customize the plot
plt.title('Top 10 neighborhoods with highest average number of drop-offs in November 2017', fontsize=16, color='r')
plt.xticks(rotation=0, fontsize=12)
plt.xlabel('Average number of drop-offs', fontsize=14, color='k')
plt.ylabel('Name of neighborhood', fontsize=14, color='k')
plt.grid(b=True, which='both', color='b', linestyle='--', alpha=0.1)
plt.minorticks_on()
plt.show()

In [ ]:
data=rides.sort_values('trips_amount', ascending=False)[:10]
data

### Conclusion
As its name refers, the **Flash Cab** company was flashing on November 15-16, 2017 with around **20k trips**. The second company was the **Taxi Affiliation Services with around 11k** trips, which less than 2/3 of the Flash Cab. Again, 11k is still very successfully when we consider that there were some companies with only several trips on that date. However, to get a better comparison, it would be great if we could compare the number of cabs each company owns. In such a case, we can clearly say which company was efficient. For example, maybe one company has only 5 cabs but performed 200 rides, while Flash Cab has 20k cabs and made 20k trips.

More than **10k taxi rides ended in the Loop**- a vibrant neighborhood of Chicago with eclectic eateries, shops, theaters and parks! Next comes the neighborhoods **River North, the Streeterville, and the West Loop - all of them have drop-offs of above 5k**. For the rest of the neighborhoods, the number of drop-offs decreases gradually. The high number of trips ending in specific neighborhoods might be closely related to the population, businesses and attraction centers in that neighborhood.

## Test hypothesis
We will check if "The average duration of rides from Loop neighborhood to O'Hare International Airport changes on rainy Saturdays" or not. For this task we are going to define our hypothesis as follow:
   - NULL hypothesis: trip durations are equal (or not significantly different) regardless of weather conditions;
   - Alternative hypothesis: trip durations differ significantly depending on weather conditions.
   
As a generally accepted limit, 95% confidence is enough for us, hence alpha=0.05

Before jumping into statistical testing, let's first get the required data from the `to_airport` dataset and visualize their distribution.

In [ ]:
#data we need is trip durations on good and bad weather conditions
good=to_airport[to_airport['weather_conditions'] == 'Good']['duration_seconds']
bad=to_airport[to_airport['weather_conditions'] == 'Bad']['duration_seconds']

#create figure object
plt.figure(figsize=(12,8))

#plot distribution curves for each weather conditions
sns.distplot(good, hist=False, kde_kws={'shade':True, 'color':'green', 'alpha':0.3}, label='Good')
sns.distplot(bad, hist=False, kde_kws={'shade':True, 'color':'red', 'alpha':0.3}, label='Bad')

#customize the figure
plt.title('Distribution of trip durations on Good and Bad weather conditions', fontsize=16, color='r')
plt.xlabel('Duration of trip (seconds)', fontsize=14, color='k')
plt.ylabel('Distribution density (a.u)', fontsize=14, color='k')
plt.xlim(0, 5000) #there are some extreme values, we don't need to show them for now
plt.legend(fontsize=14)
plt.grid(b=True, which='both', color='b', linestyle='--', alpha=0.1)
plt.minorticks_on()
plt.show()

In [ ]:
#student's t-test for comparison of two distributions
alpha=0.05 #95% confidence level

#test difference of mean of good and bad
test_result = st.ttest_ind(good, bad, equal_var=False)
print('P-value of the test is:',test_result.pvalue)

if test_result.pvalue < alpha:
    print('We reject the null hypothesis! \nAn average trip duration of {}\u00B1{} on good weather conditions is significantly \
lower than average trip duration of {}\u00B1{} on bad weather conditions.'\
          .format(round(good.mean(), 2), round(good.std(), 2), round(bad.mean(), 2), round(bad.std(), 2)))
else:
    print('We reject the null hypothesis! \nAn average trip duration of {}\u00B1{} on good weather conditions IS NOT significantly \
DIFFERENT than average trip duration of {}\u00B1{} on bad weather conditions.'\
          .format(round(good.mean(), 2), round(good.std(), 2), round(bad.mean(), 2), round(bad.std(), 2)))

### Conclusion
From the prior visualization, it has been observed that the peak of the plot of average trip durations on good weather conditions was located on the right compared to average trip durations on bad weather conditions. When the statistical analysis performed, it was seen that indeed average trip durations in good weather conditions was significantly lower than average trip durations in bad weather conditions (1999.68±759.2 vs 2427.21±721.31).

## Summary
All three dataset has been successfully loaded and been got familiarized. Datatypes of columns were correct, except the `start_ts` column of the to_airport dataset. It was in *object* type and then converted to *datetime* type. Datasets have been check for missing and duplicate values. No significant issues were observed other than duplicates in the to_airport table. However, we don't have a trip_id column, therefore we can not exactly say if these rows are duplicated or separate observations with equal values. Hence, it was decided to keep them!


As its name refers, the **Flash Cab** company was flashing on November 15-16, 2017 with around **20k trips**. The second company was the **Taxi Affiliation Services with around 11k** trips, which less than 2/3 of the Flash Cab. Again, 11k is still very successfully when we consider that there were some companies with only several trips on that date. However, to get a better comparison, it would be great if we could compare the number of cabs each company owns. In such a case, we can clearly say which company was efficient. For example, maybe one company has only 5 cabs but performed 200 rides, while Flash Cab has 20k cabs and made 20k trips.

More than **10k taxi rides ended in the Loop**- a vibrant neighborhood of Chicago with eclectic eateries, shops, theaters and parks! Next comes the neighborhoods **River North, the Streeterville, and the West Loop - all of them have drop-offs of above 5k**. For the rest of the neighborhoods, the number of drop-offs decreases gradually. The high number of trips ending in specific neighborhoods might be closely related to the population, businesses and attraction centers in that neighborhood.

When the statistical analysis performed, it was seen that the average trip durations in **good weather conditions (1999.68±759.2)** was significantly lower than average trip durations in **bad weather conditions (2427.21±721.31)**.